# Part 2:Data Models

## Importing libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

## Importing Data

In [2]:
#importing data
#df = pd.read_csv('Combined_LCA_Disclosure_Data_FY2020_to_FY2024.csv', low_memory=False)
df2020 = pd.read_csv('Combined_LCA_Disclosure_Data_FY2020.csv', low_memory=False)
df2021 = pd.read_csv('Combined_LCA_Disclosure_Data_FY2021.csv', low_memory=False)
df2022 = pd.read_csv('Combined_LCA_Disclosure_Data_FY2022.csv', low_memory=False)
df2023 = pd.read_csv('Combined_LCA_Disclosure_Data_FY2023.csv', low_memory=False)
df2024 = pd.read_csv('Combined_LCA_Disclosure_Data_FY2024.csv', low_memory=False)
all_data = [df2020, df2021,df2022,df2023,df2024]

df = pd.concat(all_data)
df.head(5)

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,...,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL,version https://git-lfs.github.com/spec/v1
0,I-200-19268-393467,Certified,2019-09-25,2019-10-01,NaN,H-1B,"APPLICATION ENGINEER, OMS [15-1199.02]",15-1199,"COMPUTER OCCUPATIONS, ALL OTHER",Y,...,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,NaN
1,I-200-19268-638983,Certified,2019-09-25,2019-10-01,NaN,H-1B,BI DEVELOPER II,15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,...,Y,BOTH,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,NaN
2,I-200-19268-177184,Certified,2019-09-25,2019-10-01,NaN,H-1B,QUALITY ENGINEER,17-2141,MECHANICAL ENGINEERS,Y,...,Y,BOTH,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,NaN
3,I-200-19268-936403,Certified,2019-09-25,2019-10-01,NaN,H-1B,"SOFTWARE DEVELOPER, APPLICATIONS",15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,...,Y,BOTH,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,NaN
4,I-200-19268-394079,Certified,2019-09-25,2019-10-01,NaN,H-1B,QUALITY ENGINEER LEVEL II,15-1199,"COMPUTER OCCUPATIONS, ALL OTHER",Y,...,Y,BOTH,NaN,Disclose Business,NaN,NaN,NaN,NaN,LEGAL@THEEGIANTS.COM,NaN


In [67]:
# Delete previous dataframes to free memory
del df2020, df2021, df2022, df2023, df2024, all_data

## Basic Data Processing 

### 1. Finding Annual Wage

In [75]:
#Salary Analysis
#Clean and convert salaries to numeric, handling outliers

unique_units = df['PW_UNIT_OF_PAY'].unique()
print(unique_units)

['Year' 'Hour' 'Bi-Weekly' nan 'Month' 'Week']


In [76]:
#"Prevailing Wage" * conversion factors to find annual wage
df['PREVAILING_WAGE'] = pd.to_numeric(df['PREVAILING_WAGE'], errors='coerce')
df = df[df['PREVAILING_WAGE'] > 0]  # Filter out non-positive values

conversion_factors = {
    'Year': 1,            # No Normalization
    'Month': 12,          # 12 months in a year
    'Bi-Weekly': 26,      # 26 bi-weekly periods in a year
    'Week': 52,           # 52 weeks in a year
    'Hour': 2080          # 52 weeks in a year x 40 hrs a week
}

df['ANNUAL_WAGE'] = df.apply(
    lambda row: row['PREVAILING_WAGE'] * conversion_factors.get(row['PW_UNIT_OF_PAY'], 1),
    axis=1
)

In [77]:
print(df[df['PW_UNIT_OF_PAY'] == 'Hour'][['PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'ANNUAL_WAGE']].head())

    PREVAILING_WAGE PW_UNIT_OF_PAY  ANNUAL_WAGE
1              39.0           Hour      81120.0
2              39.0           Hour      81120.0
3              53.0           Hour     110240.0
59             28.0           Hour      58240.0
60             29.0           Hour      60320.0


### 2. Finding Decision Duration of Applications

In [86]:
#get the processing time from subtracting decision date and start date and case status is certified
#processing the time into usable format

df['BEGIN_DATE'] = pd.to_datetime(df['BEGIN_DATE'])
df['DECISION_DATE'] = pd.to_datetime(df['DECISION_DATE'])
df['RECEIVED_DATE'] = pd.to_datetime(df['RECEIVED_DATE'])

df['Decision_Duration'] = df['BEGIN_DATE'] - df['DECISION_DATE']

df.Decision_Duration.head()

BEGIN_DATE
2024-10-01    118700
2023-10-01    114155
2022-10-01    105181
2020-10-01     89887
2021-10-01     87909
               ...  
2019-01-17         1
2019-06-26         1
2019-01-10         1
2019-01-03         1
2025-03-26         1
Name: count, Length: 2226, dtype: int64
DECISION_DATE
2020-10-15    29251
2020-12-16    13401
2023-10-06    11811
2023-11-17    11389
2020-12-17     9003
              ...  
2021-09-18        1
2022-08-21        1
2023-01-15        1
2021-12-26        1
2020-09-12        1
Name: count, Length: 1795, dtype: int64


0           6 days
1          99 days
2           2 days
3           6 days
4           8 days
            ...   
890363   -245 days
890364     56 days
890365      1 days
890366     92 days
890367    174 days
Name: Decision_Duration, Length: 3562736, dtype: timedelta64[ns]

In [87]:
#selecting features based on correlation coefficients from last section
selectdf = df.loc[:, ['ANNUAL_WAGE','SUPPORT_H1B','EMPLOYER_NAME','AGENT_ATTORNEY_CITY','WORKSITE_WORKERS','TOTAL_WORKER_POSITIONS','RECEIVED_DATE','Decision_Duration']]

In [88]:
'''
Preprocessing features
#:'Decision_Duration','TOTAL_WORKER_POSITIONS','ANNUAL_WAGE'
Categorical: 'SUPPORT_H1B','EMPLOYER_NAME','AGENT_ATTORNEY_CITY','WORKSITE_WORKERS','RECEIVED_DATE']]
'''
cat_columns = ['SUPPORT_H1B','EMPLOYER_NAME','AGENT_ATTORNEY_CITY','WORKSITE_WORKERS','RECEIVED_DATE']
for col in cat_columns:
    label_encoder = LabelEncoder()
    selectdf[col] = label_encoder.fit_transform(selectdf[col])


In [89]:
df1.head(5)

,CASE_STATUS,Decision_Duration,RECEIVED_DATE,SOC_TITLE,FULL_TIME_POSITION,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,AGENT_REPRESENTING_EMPLOYER,TOTAL_WORKER_POSITIONS,WORKSITE_CITY,WORKSITE_STATE,WORKSITE_WORKERS,WORKSITE_POSTAL_CODE,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,ANNUAL_WAGE,H_1B_DEPENDENT,SUPPORT_H1B
0,0,6 days,223,290,1,89786,5727,38,2,1,13608,37,82,11325,494,16,95118.0,0,4
1,0,99 days,223,1607,1,47200,6247,4,2,1,4637,46,82,10056,1296,9,81120.0,2,2
2,0,2 days,223,1122,1,55202,11671,24,2,1,8925,23,82,12364,2742,25,81120.0,2,2
3,0,6 days,223,1607,1,150953,13067,24,0,1,27225,23,82,12411,3155,58,110240.0,2,2
4,0,8 days,223,290,1,53537,2397,14,0,1,4515,37,82,11597,3155,58,65333.0,2,2
